In [ ]:
!cat ../data/export_23-5_minified.json | jq -c '.[]' > ../data/goodformat.json

In [11]:
!export GOOGLE_APPLICATION_CREDENTIALS=/Users/lauredegrave/code/casicoco/gcp/marbotic-310a9a33f756.json
!export GOOGLE_APPLICATION_CREDENTIALS=~/key.json
!echo $GOOGLE_APPLICATION_CREDENTIALS

/Users/lauredegrave/code/casicoco/gcp/sage-outrider-328009-2876898e70e0.json


In [2]:
import pandas as pd
from google.cloud import bigquery

project="marbotic"
credentials = "/Users/lauredegrave/code/casicoco/gcp/marbotic-310a9a33f756.json"
table_id = "marbotic_dataset.event_test"

#client = bigquery.Client(project=project, credentials=credentials)
client =bigquery.Client.from_service_account_json(credentials)

chunks = pd.read_json('../marbotic/data/goodformat.json',
                      lines=True,
                      chunksize=1000)

for c in chunks:
    print(c.columns)
    up = c[['event_id','user_properties']].copy()
    ep = c[['event_id', 'event_properties']].copy()
    c = c.drop(columns=['user_properties','event_properties'], axis=1)
    client = bigquery.Client()
    job = client.load_table_from_dataframe(c, table_id)  # Make an API request.
    job.result()  # Wait for the job to complete.
    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )
    break

Index(['city', 'client_event_time', 'country', 'device_model', 'event_id',
       'event_type', 'is_attribution_event', 'language', 'os_version',
       'region', 'session_id', 'start_version', 'user_creation_time',
       'user_id', 'user_properties', 'event_properties'],
      dtype='object')


NotFound: 404 POST https://bigquery.googleapis.com/upload/bigquery/v2/projects/sage-outrider-328009/jobs?uploadType=multipart: Not found: Dataset sage-outrider-328009:marbotic_dataset